In [20]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Embedding, GRU, Dense
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

# Sample data (Tamil to English translation)
# tamil_sentences = [
#     "எனக்கு உணவு வேண்டும்",
#     "நீங்கள் எப்படி இருக்கிறீர்கள்?",
#     "நான் நல்ல நிலையில் உள்ளேன்"
# ]

# english_sentences = [
#     "I need food",
#     "How are you?",
#     "I am fine"
# ]


tamil_file_path = '/content/data.ta1'
english_file_path = '/content/data.en1'

# Function to read sentences from a file
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines()]
    return sentences[:1000]

# Read Tamil sentences
tamil_sentences = read_sentences(tamil_file_path)

# Read English sentences
english_sentences = read_sentences(english_file_path)


# Initialize tokenizers with filters and lower options
tamil_tokenizer = Tokenizer(filters='', lower=False)
english_tokenizer = Tokenizer(filters='', lower=False)

# Fit tokenizers on the texts
tamil_tokenizer.fit_on_texts(tamil_sentences)
english_tokenizer.fit_on_texts(english_sentences)

# Add '<start>' and '<end>' tokens manually
start_token = '<start>'
end_token = '<end>'
english_tokenizer.word_index[start_token] = len(english_tokenizer.word_index) + 1
english_tokenizer.word_index[end_token] = len(english_tokenizer.word_index) + 2

# Inverse word index for later use
reverse_target_word_index = {v: k for k, v in english_tokenizer.word_index.items()}

# Convert sentences to sequences of integers
tamil_sequences = tamil_tokenizer.texts_to_sequences(tamil_sentences)
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)

# Determine vocabulary sizes
tamil_vocab_size = len(tamil_tokenizer.word_index) + 1
english_vocab_size = len(english_tokenizer.word_index) + 1

# Pad sequences
max_tamil_sequence_length = max(len(seq) for seq in tamil_sequences)
max_english_sequence_length = max(len(seq) for seq in english_sequences)

tamil_sequences = pad_sequences(tamil_sequences, maxlen=max_tamil_sequence_length, padding='post')
english_sequences = pad_sequences(english_sequences, maxlen=max_english_sequence_length, padding='post')

# Define the encoder-decoder model
def encoder_decoder_model(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size):
    # Encoder
    encoder_inputs = Input(shape=input_shape[1:])
    encoder_embedding = Embedding(tamil_vocab_size, 128)(encoder_inputs)
    encoder_gru = GRU(256, return_sequences=False, return_state=True)
    encoder_outputs, encoder_state = encoder_gru(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(output_sequence_length,))
    decoder_embedding = Embedding(english_vocab_size, 128)(decoder_inputs)
    decoder_gru = GRU(256, return_sequences=True, return_state=True)
    decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=encoder_state)
    decoder_dense = Dense(english_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Model
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy'])

    return model

# Build the model
encoder_decoder_rnn_model = encoder_decoder_model(
    tamil_sequences.shape, max_english_sequence_length, tamil_vocab_size, english_vocab_size)

# Train the model (example with dummy data)
decoder_input_data = pad_sequences(english_sequences, maxlen=max_english_sequence_length, padding='post')
decoder_target_data = pad_sequences(english_sequences[:, 0:], maxlen=max_english_sequence_length, padding='post')

encoder_decoder_rnn_model.fit(
    [tamil_sequences, decoder_input_data], decoder_target_data,
    batch_size=64, epochs=30, validation_split=0.2)

# Save the trained model
encoder_decoder_rnn_model.save_weights('encoder_decoder_model_weights.h5')

# Load the trained model weights
encoder_decoder_rnn_model.load_weights('encoder_decoder_model_weights.h5')

# Inference function
def translate_tamil_to_english(input_text):
    # Tokenize and pad input text
    input_sequence = tamil_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_tamil_sequence_length, padding='post')

    # Initialize decoder input with start token
    start_token_index = english_tokenizer.word_index[start_token]
    decoder_input = np.zeros((1, max_english_sequence_length))
    decoder_input[0, 0] = start_token_index

    # Translate using the model
    for i in range(1, max_english_sequence_length):
        output_tokens = encoder_decoder_rnn_model.predict([input_sequence, decoder_input])
        sampled_token_index = np.argmax(output_tokens[0, i, :])

        if sampled_token_index == 0:
            break

        if reverse_target_word_index[sampled_token_index] == end_token:
            break

        decoder_input[0, i] = sampled_token_index

    # Convert indices to words
    translated_sentence = [reverse_target_word_index[int(idx)] for idx in decoder_input[0] if idx != 0]
    translated_sentence = ' '.join(translated_sentence)

    return translated_sentence

# Example usage
# input_sentence = "எனக்கு உணவு வேண்டும்"
# translated_sentence = translate_tamil_to_english(input_sentence)
# print(f"Tamil: {input_sentence}")
# print(f"English: {translated_sentence}")
# encoder_decoder_rnn_model.save('encoder_decoder_model.h5')

Epoch 1/30
13/13 [==============================] - 28s 2s/step - loss: 7.9487 - accuracy: 0.5744 - val_loss: 4.2936 - val_accuracy: 0.6328
Epoch 2/30
13/13 [==============================] - 22s 2s/step - loss: 3.4036 - accuracy: 0.6193 - val_loss: 3.1813 - val_accuracy: 0.6328
Epoch 3/30
13/13 [==============================] - 24s 2s/step - loss: 3.1604 - accuracy: 0.6193 - val_loss: 3.1048 - val_accuracy: 0.6328
Epoch 4/30
13/13 [==============================] - 24s 2s/step - loss: 2.9636 - accuracy: 0.6202 - val_loss: 2.9343 - val_accuracy: 0.6352
Epoch 5/30
13/13 [==============================] - 22s 2s/step - loss: 2.8275 - accuracy: 0.6243 - val_loss: 2.8773 - val_accuracy: 0.6398
Epoch 6/30
13/13 [==============================] - 23s 2s/step - loss: 2.7769 - accuracy: 0.6261 - val_loss: 2.8519 - val_accuracy: 0.6392
Epoch 7/30
13/13 [==============================] - 22s 2s/step - loss: 2.7387 - accuracy: 0.6267 - val_loss: 2.8354 - val_accuracy: 0.6402
Epoch 8/30
13/13 [==

In [22]:
input_sentence = "இருவித ஊதிய விகிதங்களும் ஏற்கப்பட்டன மேற்கு ஐரோப்பிய துறைமுகப் பணியாளர்களுக்கும் கிழக்கு ஐரோப்பிய நாடுகள் , ஆசியா இவற்றின் தொழிலாளர்களுக்கும் இடையே பிளவுகளைக் கொண்டுவரும் துறைமுகத் தொழிற்சங்கங்களின் கொள்கை இதே பேரழிவு விளைவுகளைத்தான் கொண்டுவரும்."
translated_sentence = translate_tamil_to_english(input_sentence)
print(f"Tamil: {input_sentence}")
print(f"English: {translated_sentence}")


1/1 [==============================] - 0s 247ms/step
Tamil: இருவித ஊதிய விகிதங்களும் ஏற்கப்பட்டன மேற்கு ஐரோப்பிய துறைமுகப் பணியாளர்களுக்கும் கிழக்கு ஐரோப்பிய நாடுகள் , ஆசியா இவற்றின் தொழிலாளர்களுக்கும் இடையே பிளவுகளைக் கொண்டுவரும் துறைமுகத் தொழிற்சங்கங்களின் கொள்கை இதே பேரழிவு விளைவுகளைத்தான் கொண்டுவரும்.
English: <start> this were were .


In [18]:
print(tamil_sentences[10])
print(english_sentences[10])

இருவித ஊதிய விகிதங்களும் ஏற்கப்பட்டன மேற்கு ஐரோப்பிய துறைமுகப் பணியாளர்களுக்கும் கிழக்கு ஐரோப்பிய நாடுகள் , ஆசியா இவற்றின் தொழிலாளர்களுக்கும் இடையே பிளவுகளைக் கொண்டுவரும் துறைமுகத் தொழிற்சங்கங்களின் கொள்கை இதே பேரழிவு விளைவுகளைத்தான் கொண்டுவரும் .
the policy of the port unions that allows a division between western european dockworkers and those from the eastern european states and asia will have the same disastrous consequences .
